In [1]:
from joblib import Parallel, delayed
from pyaspeller import YandexSpeller
from pymystem3 import Mystem
import os
import pandas as pd
import dask.dataframe as dd
from tqdm import tqdm
speller = YandexSpeller()
mystem = Mystem(grammar_info=False)

In [2]:
df = pd.read_csv('./docs.tsv', sep='\t', header=None)

In [3]:
df.columns = ['Id', 'Title', 'Text']

In [4]:
df.tail()

,Id,Title,Text
582162,582162,УРОК 2 КАК ПОЛУЧАТЬ БОЛЬШЕ ЗАКАЗОВ РЕЙТИНГ ЯНД...,SKIP TO CONTENT СЛЕДИТЕ ЗА РЕЙТИНГОМ РАСПРЕДЕЛ...
582163,582163,NaN,SAN ANTONIO WEB DESIGN HOME ABOUT MINISTRIES T...
582164,582164,0C5V5EY INSURANCE ARCHIVE STATE OF SOUTH AFRIC...,STATE OF SOUTH AFRICAN POLITICS TODAY YESTERDA...
582165,582165,488204_120826113120_D5EVO,488204_120826113120_D5EVO 17 09 2013 BY ADMIN ...
582166,582166,SALE OF MOBILE SIM CARDS AT ECONET SHOPS I PAI...,I PAID A BRIBE INICIAR SESIÓN CORREO ELECTRÓNI...


### BM25

In [5]:
queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']

In [6]:
queries

,Id,Query
0,0,13 причин почему
1,1,1 положительный и 1 отрицательный могут ли
2,2,2016 действует ли зао рождественская мануфактура
3,3,1 месяц после операции на кишечнике диета что ...
4,4,2 правды 1 ложь что можно придумать
...,...,...
6306,6306,является ли тойота харриер внедорожником
6307,6307,як можно очистить крейду
6308,6308,являются ли реактивы медицинскими изделиями
6309,6309,являются ли словообразовательными парами слова...


In [7]:
queries['Id'] = queries['Id'].apply(lambda x: int(x))
queries['Query'] = queries['Query'].apply(lambda x: x.upper())

In [8]:
queries

,Id,Query
0,0,13 ПРИЧИН ПОЧЕМУ
1,1,1 ПОЛОЖИТЕЛЬНЫЙ И 1 ОТРИЦАТЕЛЬНЫЙ МОГУТ ЛИ
2,2,2016 ДЕЙСТВУЕТ ЛИ ЗАО РОЖДЕСТВЕНСКАЯ МАНУФАКТУРА
3,3,1 МЕСЯЦ ПОСЛЕ ОПЕРАЦИИ НА КИШЕЧНИКЕ ДИЕТА ЧТО ...
4,4,2 ПРАВДЫ 1 ЛОЖЬ ЧТО МОЖНО ПРИДУМАТЬ
...,...,...
6306,6306,ЯВЛЯЕТСЯ ЛИ ТОЙОТА ХАРРИЕР ВНЕДОРОЖНИКОМ
6307,6307,ЯК МОЖНО ОЧИСТИТЬ КРЕЙДУ
6308,6308,ЯВЛЯЮТСЯ ЛИ РЕАКТИВЫ МЕДИЦИНСКИМИ ИЗДЕЛИЯМИ
6309,6309,ЯВЛЯЮТСЯ ЛИ СЛОВООБРАЗОВАТЕЛЬНЫМИ ПАРАМИ СЛОВА...


In [9]:
df['Id'] = df['Id'].apply(lambda x: int(x))

In [10]:
df.tail(25)

,Id,Title,Text
582142,582142,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА НА ГЛАВНУЮ ...
582143,582143,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА НА ГЛАВНУЮ ...
582144,582144,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,РОССИЯ И УКРАИНА ЛЕНИНГРАДСКАЯ ОБЛ Г САНКТ ПЕТ...
582145,582145,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,РОССИЯ И УКРАИНА ВЛАД ХЛАДОКОМБИНАТ 2 АДРЕС ПР...
582146,582146,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА НА ГЛАВНУЮ ...
582147,582147,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА НА ГЛАВНУЮ ...
582148,582148,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА,АДРЕСА И ТЕЛЕФОНЫ РОССИЯ И УКРАИНА НА ГЛАВНУЮ ...
582149,582149,КУНГ ФУ ПАНДА 3 2016 KINOPARK,ГЛАВНАЯ КАТАЛОГ НОВИНКИ СЕРИАЛЫ МУЛЬТФИЛЬМЫ ГЛ...
582150,582150,КАК ВЗЛОМАТЬ QIWI КОШЕЛЕК БЕСПЛАТНАЯ ПРОГРАММА...,LOADING TOGGLE NAVIGATION VLCLIP XYZ YOUTUBE F...
582151,582151,КАК ПРОЧИТАТЬ ЧУЖИЕ СООБЩЕНИЯ В ВИБЕРЕ ВАЙБЕР ...,ПЕРЕЙТИ К СОДЕРЖИМОМУ ВОЙТИ РЕГИСТРАЦИЯ ПОИСК ...


In [11]:
df.drop(['Title'], axis=1, inplace=True)

In [50]:
import math
from tqdm import tqdm
import numpy as np
from multiprocessing import Pool, cpu_count

"""
All of these algorithms have been taken from the paper:
Trotmam et al, Improvements to BM25 and Language Models Examined
Here we implement all the BM25 variations mentioned. 
"""


class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = 582167
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for docu in tqdm(corpus):
            document = str(docu).split(" ")[:512]
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

        self.avgdl = num_doc / self.corpus_size
        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        top_n = np.argsort(scores)[::-1][:n]
        return [documents[i] for i in top_n]


class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()


class BM25Plus(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, delta=1):
        # Algorithm specific parameters
        self.k1 = k1
        self.b = b
        self.delta = delta
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        for word, freq in nd.items():
            idf = math.log((self.corpus_size + 1) / freq)
            self.idf[word] = idf

    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (self.delta + (q_freq * (self.k1 + 1)) /
                                               (self.k1 * (1 - self.b + self.b * doc_len / self.avgdl) + q_freq))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (self.delta + (q_freq * (self.k1 + 1)) /
                                               (self.k1 * (1 - self.b + self.b * doc_len / self.avgdl) + q_freq))
        return score.tolist()

In [22]:
from guppy import hpy; h=hpy()

In [36]:
h.heap()

Partition of a set of 731068 objects. Total size = 215653445646 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      6   0 115305856335  53 115305856335  53 pandas.core.frame.DataFrame
     1     11   0 100257146238  46 215563002573 100 pandas.core.series.Series
     2 278917  38 30990557   0 215593993130 100 str
     3 142476  19 12293440   0 215606286570 100 tuple
     4  12808   2  5607208   0 215611893778 100 dict (no owner)
     5  63279   9  4894090   0 215616787868 100 bytes
     6    111   0  4771062   0 215621558930 100 numpy.ndarray
     7  32160   4  4656272   0 215626215202 100 types.CodeType
     8  28918   4  4164192   0 215630379394 100 function
     9   3354   0  3257352   0 215633636746 100 type
<1162 more rows. Type e.g. '_.more' to view.>

In [51]:
bm25_Okapi = BM25Okapi(df['Text'])
bm25_Plus = BM25Plus(df['Text'])

100%|██████████| 582167/582167 [10:01<00:00, 967.27it/s] 


In [52]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

In [53]:
marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

In [54]:
sample_df['BM25Okapi'] = None
sample_df['BM25Plus'] = None

In [55]:
sample_df

,query_id,url_id,BM25Okapi,BM25Plus
0,0,340485,None,None
1,0,68106,None,None
2,0,237314,None,None
3,0,203791,None,None
4,0,53265,None,None
...,...,...,...,...
202074,6305,63981,None,None
202075,6305,354802,None,None
202076,6305,275960,None,None
202077,6305,338427,None,None


In [56]:
max(sample_df['url_id'])

582166

In [57]:
queries

,Id,Query
0,0,13 ПРИЧИН ПОЧЕМУ
1,1,1 ПОЛОЖИТЕЛЬНЫЙ И 1 ОТРИЦАТЕЛЬНЫЙ МОГУТ ЛИ
2,2,2016 ДЕЙСТВУЕТ ЛИ ЗАО РОЖДЕСТВЕНСКАЯ МАНУФАКТУРА
3,3,1 МЕСЯЦ ПОСЛЕ ОПЕРАЦИИ НА КИШЕЧНИКЕ ДИЕТА ЧТО ...
4,4,2 ПРАВДЫ 1 ЛОЖЬ ЧТО МОЖНО ПРИДУМАТЬ
...,...,...
6306,6306,ЯВЛЯЕТСЯ ЛИ ТОЙОТА ХАРРИЕР ВНЕДОРОЖНИКОМ
6307,6307,ЯК МОЖНО ОЧИСТИТЬ КРЕЙДУ
6308,6308,ЯВЛЯЮТСЯ ЛИ РЕАКТИВЫ МЕДИЦИНСКИМИ ИЗДЕЛИЯМИ
6309,6309,ЯВЛЯЮТСЯ ЛИ СЛОВООБРАЗОВАТЕЛЬНЫМИ ПАРАМИ СЛОВА...


In [58]:
for query in tqdm(set(sample_df['query_id'])):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]
    tokenized_query = q.split(" ")

    bm25_1 = bm25_Okapi.get_batch_scores(tokenized_query, docs_id)
    bm25_3 = bm25_Plus.get_batch_scores(tokenized_query, docs_id)
    
    sample_df.loc[sample_df['query_id'] == query, 'BM25Okapi'] = bm25_1
    #sample_df.loc[sample_df['query_id'] == query, 'BM25L'] = bm25_2
    sample_df.loc[sample_df['query_id'] == query, 'BM25Plus'] = bm25_3

100%|██████████| 6311/6311 [18:55<00:00,  5.56it/s]


In [59]:
sample_df.to_csv('./BM25.tsv',sep='\t',index=False)

In [60]:
sample_df

,query_id,url_id,BM25Okapi,BM25Plus
0,0,340485,12.759658,20.659241
1,0,68106,10.986106,18.83912
2,0,237314,9.378046,17.125068
3,0,203791,12.913466,20.893519
4,0,53265,15.188733,23.335037
...,...,...,...,...
202074,6305,63981,26.528689,37.009709
202075,6305,354802,7.781366,16.878479
202076,6305,275960,27.964711,38.593277
202077,6305,338427,32.54708,42.501235


In [65]:
del bm25_Okapi

In [66]:
del bm25_Plus